In [ ]:
import pandas as pd
import altair as alt
import ipywidgets as widgets
from ipywidgets import Dropdown, VBox, HBox, HTML, Output
from IPython.display import display


alt.data_transformers.disable_max_rows()
alt.data_transformers.enable("vegafusion")

columns_to_keep = ['country_name',
                   'year',
				   'e_regionpol_7C',
				   'e_regiongeo',

                   'v2x_polyarchy',
                   'v2x_libdem',
                   'v2x_partipdem',
                   'v2x_delibdem',
                   'v2x_egaldem',

                   'v2elgvsuflvl',
				   'v2expathhg',
                   'v2ddlexci',
                   'v2elrstrct',
                   'v2ddcredal',
                   'v2exfemhog',
                   'v2elcomvot',
                   'v2elfemrst',

				   'v2xca_academ',
				   'v2cafexch',
				   'v2cafres',
				   'v2cainsaut',
				   'v2casurv',
				   'v2clacfree', 

				    'v2mecenefm',
        			'v2mecenefi',
					'v2merange',
					'v2mebias',
					'v2xcl_disc',
					'v2x_freexp',
     
					'v2smgovfilcap',
					'v2smgovfilprc',
					'v2smgovshutcap',
					'v2smgovshut',
					'v2smgovsm',
					'v2smgovsmalt',
					'v2smgovsmmon',
					'v2smgovsmcenprc',
					'v2smgovcapsec',
					'v2smpolcap'

				   ]

data = pd.read_csv(
    "../../../data/raw/V-Dem-CY-Full+Others-v15.csv",
    usecols=columns_to_keep,   
    low_memory=False
)

data['year_dt'] = pd.to_datetime(data['year'], format = '%Y')

dict_for_renaming = {

	# General attributes 
    "country_name": "Country",
    "year_dt": "Year_DT",
    "year": "Year",
	"e_regionpol_7C": "Politico-Geographical Region",
	"e_regiongeo": "Region",

    # High-level indices
    "v2x_polyarchy": "Electoral Democracy Index",
    "v2x_libdem": "Liberal Democracy Index",
    "v2x_partipdem": "Participatory Democracy Index",
    "v2x_delibdem": "Deliberative Democracy Index",
    "v2x_egaldem": "Egalitarian Democracy Index",


    # Social Media Controls
    "v2smgovsm": "Gov. Social Media Shutdown in Practice",
    "v2smgovsmalt": "Gov. Social Media Alternatives (Gov-Backed Platforms)",
    "v2smgovsmmon": "Gov. Social Media Monitoring",
    "v2smgovsmcenprc": "Gov. Social Media Censorship in Practice",
	
}

data = data.rename(columns = dict_for_renaming)
data["Year"] = data["Year"].replace("", pd.NA)

high_level_indices = [
    "Electoral Democracy Index",
    "Liberal Democracy Index",
    "Participatory Democracy Index",
    "Deliberative Democracy Index",
    "Egalitarian Democracy Index"
]

social_media_attributes = [
    "Gov. Social Media Shutdown in Practice",
    "Gov. Social Media Alternatives (Gov-Backed Platforms)",
    "Gov. Social Media Monitoring",
    "Gov. Social Media Censorship in Practice",
]

data["High_Level_Mean"] = data[high_level_indices].mean(axis=1)

data

In [ ]:
needed_cols = [
    "Country",
    "Year",
    "Electoral Democracy Index",
    "Liberal Democracy Index",
    "Participatory Democracy Index",
    "Deliberative Democracy Index",
    "Egalitarian Democracy Index",
    "Gov. Social Media Shutdown in Practice",
    "Gov. Social Media Censorship in Practice",
    "Gov. Social Media Monitoring",
]

data = data[needed_cols].copy()

In [ ]:
tol10 = [
    "#6699cc",  
    "#99ddff",  
    "#44bb99",  
    "#aa4499",  
    "#aaaa00",  
    "#dddddd",  
]

data_full = data.copy()

interesting_countries = [
    "North Korea", "China", "Russia", "Iran", "Turkey", "Nigeria", "United States",
    "Canada", "Denmark", "Ethiopia", "Burma/Myanmar", "Turkmenistan",
    "Sudan", "Uganda", "United States", "Eritrea", "Bangladesh", "Equatorial Guinea"
    "United Kingdom", "South Korea", "Brazil", "Ethiopea"
]

data_line = data_full[data_full["Country"].isin(interesting_countries)].copy()

high_level_indices = [
    "Electoral Democracy Index",
    "Liberal Democracy Index",
    "Participatory Democracy Index",
    "Deliberative Democracy Index",
    "Egalitarian Democracy Index",
]

data_line["High_Level_Mean"] = data_line[high_level_indices].mean(axis=1)

year_scale = alt.Scale(domain=[2000, 2024])
country_options = sorted(data_line["Country"].dropna().unique().tolist())


def make_shutdown_barchart_2024():
    
    df_2024 = data_full[data_full["Year"] == 2024].copy()

    shutdown_chart = (
        alt.Chart(df_2024, width=500, height=150)
        .transform_aggregate(
            ShutdownScore="mean(Gov. Social Media Shutdown in Practice)",
            groupby=["Country"],
        )
        .transform_window(
            rank="rank(ShutdownScore)",
            sort=[alt.SortField("ShutdownScore", order="ascending")],
        )
        .transform_filter("datum.rank <= 10") 
        .mark_bar(cornerRadiusTopLeft=4, cornerRadiusBottomLeft=4)
        .encode(
            y=alt.Y(
                "Country:N",
                sort=alt.SortField("ShutdownScore", order="ascending"),
                title="Country",
                axis=alt.Axis(
                    labelFontSize=12,
                    titleFontSize=14,
                    titlePadding = 30,
                    offset = 0)
            ),
            x=alt.X(
                "ShutdownScore:Q",
                title="Shutdown Score (2024)",
                axis=alt.Axis(
                    labelFontSize=12,
                    titleFontSize=14,
                    titlePadding = 20,
                    offset = 0)
            ),
            color=alt.value("#ff958a"),
            tooltip=[
                alt.Tooltip("Country:N"),
                alt.Tooltip("ShutdownScore:Q", title="Shutdown Score"),
            ],
        )
        .properties(
            padding={"left": 100, "top": 0, "right": 0, "bottom": 0},
            title=alt.TitleParams(
                text="Worst Countries by Shutdown Score (2024)",
                anchor="middle",
                fontSize=14,
                offset=20,
            )
        )
    )

    return shutdown_chart



def make_dashboard(selected_country: str, show_components: bool):

    country_data = data_line[data_line["Country"] == selected_country]


    social_media_vars = [
        "Gov. Social Media Shutdown in Practice",
        "Gov. Social Media Censorship in Practice",
        "Gov. Social Media Monitoring",
        "Gov. Social Media Alternatives (Gov-Backed Platforms)",
    ]

    social_media_expr = (
        "datum.IndicatorLong == 'Gov. Social Media Monitoring' ? 'Monitoring' : "
        "datum.IndicatorLong == 'Gov. Social Media Censorship in Practice' ? 'Censorship' : "
        "datum.IndicatorLong == 'Gov. Social Media Shutdown in Practice' ? 'Shutdown' : "
        "datum.IndicatorLong == 'Gov. Social Media Alternatives (Gov-Backed Platforms)' ? "
        "'Alternatives' : 'Other'"
    )

    social_media_domain = ["Monitoring", "Censorship", "Shutdown"] 
    
    social_media_colors = [
        "#fcba03",
        "#ffaabb",
        "#ee8866",
        "#999999",
    ]

    social_media_chart = (
        alt.Chart(country_data, width=750, height=160)
        .transform_filter(alt.datum.Year >= 1990)
        .transform_fold(social_media_vars, as_=["IndicatorLong", "Score"])
        .transform_calculate(Indicator=social_media_expr)
        .mark_line(point=True, strokeWidth=2)
        .encode(
            x=alt.X(
                "Year:Q",
                scale=year_scale,
                axis=alt.Axis(
                    format="d", labelFontSize=12, titleFontSize=14, titlePadding=20
                ),
            ),
            y=alt.Y(
                "Score:Q",
                axis=alt.Axis(
                    labelFontSize=12,
                    titleFontSize=14,
                    titlePadding=20,
                    title="Score (less restrictive →)",
                ),
            ),
            color=alt.Color(
                "Indicator:N",
                title="Controls",
                scale=alt.Scale(domain=social_media_domain, range=social_media_colors),
            ),
            tooltip=["Country:N", "Year:Q", "Indicator:N", "Score:Q"],
            strokeDash=alt.condition(
                alt.datum.Indicator == "Monitoring",
                alt.value([5, 5]),   # dashed
                alt.value([0, 0])    # solid for all others
            )
        )
        .properties(title="Government Control of Social Media")
    )

    democracy_base = (
        alt.Chart(country_data, width=750, height=160)
        .transform_filter(alt.datum.Year >= 2000)
    )

    democracy_mean = (
        democracy_base.mark_area(color=tol10[5])
        .encode(
            x=alt.X(
                "Year:Q",
                scale=year_scale,
                axis=alt.Axis(format="d", labelFontSize=12, titleFontSize=14),
            ),
            y=alt.Y(
                "High_Level_Mean:Q",
                axis=alt.Axis(labelFontSize=12, titleFontSize=14, titlePadding=20),
                title="Index Value",
            ),
            opacity=alt.value(0.55),
            tooltip=[
                alt.Tooltip("Country:N"),
                alt.Tooltip("Year:Q", format="d", title="Year"),
                alt.Tooltip("High_Level_Mean:Q", title="Mean index value", format=".3f"),
            ]
        )
    )

    dimension_short_domain = [
        "Electoral",
        "Liberal",
        "Participatory",
        "Deliberative",
        "Egalitarian",
    ]
    dimension_colors = [tol10[2], tol10[0], tol10[1], tol10[3], tol10[4]]

    democracy_detail = (
        democracy_base
        .transform_fold(high_level_indices, as_=["DimensionLong", "Score"])
        .transform_calculate(
            Dimension=(
                "datum.DimensionLong == 'Electoral Democracy Index' ? 'Electoral' : "
                "datum.DimensionLong == 'Liberal Democracy Index' ? 'Liberal' : "
                "datum.DimensionLong == 'Participatory Democracy Index' ? 'Participatory' : "
                "datum.DimensionLong == 'Deliberative Democracy Index' ? 'Deliberative' : "
                "datum.DimensionLong == 'Egalitarian Democracy Index' ? 'Egalitarian' : ''"
            )
        )
        .mark_line(point=True, strokeWidth=2)
        .encode(
            x=alt.X(
                "Year:Q",
                scale=year_scale,
                axis=alt.Axis(format="d", labelFontSize=12, titleFontSize=14),
            ),
            y=alt.Y(
                "Score:Q",
                axis=alt.Axis(labelFontSize=12, titleFontSize=14),
            ),
            color=alt.Color(
                "Dimension:N",
                scale=alt.Scale(
                    domain=dimension_short_domain, range=dimension_colors
                ),
                title="Component Indices",
            ),
            tooltip=[
                alt.Tooltip("Country:N"),
                alt.Tooltip("Year:Q", format="d", title="Year"),
                alt.Tooltip("Dimension:N", title="Index"),
                alt.Tooltip("Score:Q", title="Index value", format=".3f"),
            ],
        )
    )

    if show_components:
        democracy_chart = (democracy_mean + democracy_detail).properties(
            title="Composite Democracy Index (with component scores)"
        )
    else:
        democracy_chart = democracy_mean.properties(
            title="Composite Democracy Index"
        )

    return (
        alt.vconcat(
            democracy_chart,
            social_media_chart,
            spacing=40,
        )
        .resolve_scale(color="independent")
        .properties(padding={"left": 40})
    )


instructions_html = HTML(
    "<div style='font-size:14px; text-align:left; "
    "padding-left:40px; padding-top: 10px; margin-top:10px; margin-bottom:5px;'>"
    "<b>Select</b> a country in the dropdown to explore relationship between democratic quality and government control of social media.<br>"
    "Use the checkbox to show or hide the scores averaged to form the composite democracy index."
    "</div>"
)

title_widget = HTML(
    "<h3 style='margin-bottom:50px; margin-top:200px; font-size:37px; "
    "text-align:middle; padding-left:40px;'>Democracy & Government Control of Social Media</h2>"
)

default_country = "Canada" if "Canada" in country_options else country_options[0]

country_dropdown = Dropdown(
    options=country_options,
    value=default_country,
    description="Country:",
)
country_dropdown.layout = widgets.Layout(margin="10px 15px")

show_components_checkbox = widgets.Checkbox(
    value=False,
    description="Show component democracy indices",
    indent=False,
)
show_components_checkbox.layout = widgets.Layout(margin="10px 15px 25px")

out_top = Output()
out_bottom = Output()

def update_dashboard(change=None):
    with out_top:
        out_top.clear_output()
        display(make_shutdown_barchart_2024())

    with out_bottom:
        out_bottom.clear_output()
        display(make_dashboard(
            country_dropdown.value,
            show_components_checkbox.value
        ))

country_dropdown.observe(update_dashboard, names="value")
show_components_checkbox.observe(update_dashboard, names="value")

main = VBox([
    title_widget,
    out_top,
    instructions_html,
    widgets.HBox([country_dropdown, show_components_checkbox]),
    out_bottom,
])

main.layout = widgets.Layout(margin="0 0 0 100px")
display(main)

update_dashboard()
